In [1]:
# loading and normalizing cifar10
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [17]:
# import cifar10 from keras
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

type(x_train)
type(y_train)

print(x_train.shape) # need to conver this to [50000, 3, 32, 32] for torch tensors
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


# add train_val split

In [18]:
# change to torch tensor
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)


print(x_train.size())
print(x_test.size())
print(x_train.type())
print(y_train.type())

torch.Size([50000, 32, 32, 3])
torch.Size([10000, 32, 32, 3])
torch.ByteTensor
torch.ByteTensor


In [33]:
#x_train = x_train.type('torch.DoubleTensor')
#x_test = x_test.type('torch.DoubleTensor')
#y_train = y_train.type('torch.DoubleTensor')
#y_test = y_test.type('torch.DoubleTensor')
x_train = x_train.float()
x_test = x_test.float()
#y_train = y_train.long()
#y_test = y_test.long()
y_train = y_train.squeeze(1) # labels need to be squeezed/unraveled
y_test = y_test.squeeze(1)

In [28]:
print(y_test)

tensor([[ 3],
        [ 8],
        [ 8],
        ...,
        [ 5],
        [ 1],
        [ 7]])


In [35]:
# checking shape and type
print(x_train.size())
print(x_test.size())
print(y_train.size())
print(y_test.size())
print(x_train.type())
print(x_test.type())
print(y_train.type())
print(y_test.type())

torch.Size([50000, 3, 32, 32])
torch.Size([10000, 3, 32, 32])
torch.Size([50000])
torch.Size([10000])
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.LongTensor


In [21]:
#reshaping torch tensor
x_train = x_train.permute(0,3,1,2)
x_test = x_test.permute(0,3,1,2)


In [22]:
# check tensor shape
# should be [n, channels, x, y]
print(x_train.size())
print(x_test.size())

torch.Size([50000, 3, 32, 32])
torch.Size([10000, 3, 32, 32])


In [36]:
# change into torch dataset

from torch.utils.data import TensorDataset

train = TensorDataset(x_train, y_train)

test = TensorDataset(x_test, y_test)

In [37]:
# setting up a train and test loader
trainloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)


In [38]:
# creating NN

# set zero padding to (k-1)/2 for stride = 1
# 
import torch.nn as nn
import torch.nn.functional as F

class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()
        self.conv_block1 = nn.Sequential(
                            nn.Conv2d(3, 8, 5, padding=2), # [in-channel, out-channel(# of filters), kernel size (5,5)]
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
        )
        self.conv_block2 = nn.Sequential(
                            nn.Conv2d(8, 8, 5, padding = 2), # note that in-channel for the next layer is the same as out-channel from prev conv layer
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
        )
        self.fc1 = nn.Linear(512, 32) # 8*5*5 is flattened 8 filters and 5x5 kernel size
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x.view(x.size(0), -1) # flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
        
net = convnet()

In [39]:
# define loss and optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters()) #net.parameters

In [40]:
# model training
n_epochs = 2

for epoch in range(n_epochs):
    running_loss = 0.0
    for i, train_data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = train_data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

Finished Training


In [41]:
# testing
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 27 %
